# Package

In [ ]:
# !pip install -U sentence-transformers
!pip install sentence-transformers==2.7.0
!pip install rank-bm25

!pip install sastrawi
!pip install swifter

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

# Load Library

In [ ]:
from sentence_transformers import SentenceTransformer,util
import pandas as pd
import pickle

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

import string
import re

from statistics import mean

from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [ ]:
def text_prepros(text):

    # converting to lowercase
    text = text.lower()
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"") #tab #new_line #unicode #\
    # remove puctuation
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # remove whitespace leading & trailing
    text = text.strip()
    # substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # remove number
    text = re.sub(r'\d+', ' ', text)
    return text

def word_tokenize_wrapper(text):
    return word_tokenize(text)

# stem function bag of word
def stemmed_wrapper(term):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(term)


def set_sumber(state, pasal, ayat, angka, huruf, paragraf):
  sumber = ''
  sumber += state
  sumber += ' - Psl' + pasal
  if ayat != '0':
    sumber += '/Ayt' + ayat
  if angka != '0':
    sumber += '/Ank' + angka
  if huruf != '0':
    sumber += '/Hrf' + huruf
  if paragraf != '0':
    sumber += '/Par' + paragraf
  return sumber

def search_term(seris, term):
  index_term = []
  i = 0
  for seri in seris:
    if term in seri:
      index_term.append(i)
    i += 1
  return index_term

# Dataset

In [ ]:
dfAns = pd.read_excel('/content/drive/MyDrive/Thesis/Dataset KUP.xlsx', dtype={'Angka':'str','Paragraf':'str'})
dfAns.drop('Isi', axis=1, inplace=True)
dfAns.rename(columns={'Isi_cleaned':'Isi'}, inplace=True)
dfAns['Isi_lower'] = dfAns['Isi'].apply(lambda x: x.lower())

# Preprocessing

In [ ]:
dfAns['Isi_Processed'] = dfAns['Isi'].apply(text_prepros)
dfAns['Isi_Tokenized'] = dfAns['Isi_Processed'].apply(word_tokenize_wrapper)
dfAns['Isi_Tokenized_Joined'] = dfAns['Isi_Tokenized'].apply(lambda x: ' '.join(x))

# stemming
# dfAns['Isi_Stemmed'] = dfAns['Isi_Tokenized_Joined'].swifter.apply(stemmed_wrapper)
# dfAns.to_csv('/content/drive/MyDrive/Thesis/Dataset KUP Preprocessed.csv', sep=';', index=False)

# Load Preprocessed Dataset

In [ ]:
dfAns = pd.read_csv('/content/drive/MyDrive/Thesis/Dataset KUP Preprocessed.csv', sep=';', dtype={'Angka':'str','Paragraf':'str'})

# Encoding

https://stackoverflow.com/questions/68334844/how-to-save-sentence-bert-output-vectors-to-a-file

## Multilingual-MPnet

In [ ]:
# !unzip /content/drive/MyDrive/Thesis_Finetuned/Final_Model_MPNET.zip -d /content/drive/MyDrive/Thesis_Finetuned/Final_Model_MPNET

In [ ]:
# model_multimpnet = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

model_folder = ''; model_name = 'fine_tuned_model_kup_all_a1'
# model_folder = ''; model_name = 'fine_tuned_model_kup_all_a2'
# model_folder = ''; model_name = 'fine_tuned_model_kup_all_a3'
model_multimpnet = SentenceTransformer("/content/drive/MyDrive/Thesis_Finetuned/"+model_folder+"/"+model_name)

# multilingual_mpnet_embeddings_isi = model_multimpnet.encode(dfAns['Isi'])
# with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+"_isi.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_isi': multilingual_mpnet_embeddings_isi},fOut)

# multilingual_mpnet_embeddings = model_multimpnet.encode(dfAns['Isi_lower'])
# with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+".pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings': multilingual_mpnet_embeddings},fOut)

# multilingual_mpnet_embeddings_joined = model_multimpnet.encode(dfAns['Isi_Tokenized_Joined'])
# with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+"_joined.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_joined': multilingual_mpnet_embeddings_joined},fOut)

# multilingual_mpnet_embeddings_stemmed = model_multimpnet.encode(dfAns['Isi_Stemmed'])
# with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+"_stemmed.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_stemmed': multilingual_mpnet_embeddings_stemmed},fOut)

with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+"_isi.pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_isi = cache_data['multilingual_mpnet_embeddings_isi']

with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_"+model_name+".pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings = cache_data['multilingual_mpnet_embeddings']

with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_joined_"+model_name+".pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_joined = cache_data['multilingual_mpnet_embeddings_joined']

with open("/content/drive/MyDrive/Thesis_Finetuned/multilingual_mpnet_embeddings_stemmed_"+model_name+".pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_stemmed = cache_data['multilingual_mpnet_embeddings_stemmed']

In [ ]:
# model_multimpnet_raw = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model_multimpnet_raw = SentenceTransformer("/content/drive/MyDrive/Thesis_Finetuned/paraphrase-multilingual-mpnet-base-v2")

# multilingual_mpnet_embeddings_isi_raw = model_multimpnet.encode(dfAns['Isi'])
# with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_isi_raw.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_isi_raw': multilingual_mpnet_embeddings_isi_raw},fOut)

# multilingual_mpnet_embeddings_raw = model_multimpnet_raw.encode(dfAns['Isi_lower'])
# with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_raw.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_raw': multilingual_mpnet_embeddings_raw},fOut)

# multilingual_mpnet_embeddings_joined_raw = model_multimpnet_raw.encode(dfAns['Isi_Tokenized_Joined'])
# with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_joined_raw.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_joined_raw': multilingual_mpnet_embeddings_joined_raw},fOut)

# multilingual_mpnet_embeddings_stemmed_raw = model_multimpnet_raw.encode(dfAns['Isi_Stemmed'])
# with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_stemmed_raw.pkl", "wb") as fOut:
#     pickle.dump({'multilingual_mpnet_embeddings_stemmed_raw': multilingual_mpnet_embeddings_stemmed_raw},fOut)

with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_isi_raw.pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_isi_raw = cache_data['multilingual_mpnet_embeddings_isi_raw']

with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_raw.pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_raw = cache_data['multilingual_mpnet_embeddings_raw']

with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_joined_raw.pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_joined_raw = cache_data['multilingual_mpnet_embeddings_joined_raw']

with open("/content/drive/MyDrive/Thesis/multilingual_mpnet_embeddings_stemmed_raw.pkl", "rb") as fIn:
  cache_data = pickle.load(fIn)
  multilingual_mpnet_embeddings_stemmed_raw = cache_data['multilingual_mpnet_embeddings_stemmed_raw']

In [ ]:
print(dfAns.loc[1, 'Isi'])
print('multilingual_mpnet_embeddings_isi_raw:', len(multilingual_mpnet_embeddings_isi_raw[1]), '\n', multilingual_mpnet_embeddings_isi_raw[1][:24])
print(dfAns.loc[1, 'Isi_lower'])
print('multilingual_mpnet_embeddings_raw:\n', multilingual_mpnet_embeddings_raw[1][:24])
print(dfAns.loc[1, 'Isi_Tokenized_Joined'])
print('multilingual_mpnet_embeddings_joined_raw:\n', multilingual_mpnet_embeddings_joined_raw[1][:24])
print('multilingual_mpnet_embeddings_stemmed_raw:\n', multilingual_mpnet_embeddings_stemmed_raw[1][:24])

Pajak adalah kontribusi wajib kepada negara yang terutang oleh orang pribadi atau badan yang bersifat memaksa berdasarkan Undang-Undang, dengan tidak mendapatkan imbalan secara langsung dan digunakan untuk keperluan negara bagi sebesar-besarnya kemakmuran rakyat.
multilingual_mpnet_embeddings_isi_raw: 768 
 [ 0.0251285   0.11086553 -0.01207943  0.02515922 -0.12136071 -0.03784876
  0.07196373  0.00782752  0.11384453  0.07801113  0.0326741   0.05447559
  0.08495865 -0.29741096  0.08755189 -0.10016664  0.01935305  0.10326253
 -0.01948993  0.09886288  0.05033723  0.07653238  0.00264413  0.07187161]
pajak adalah kontribusi wajib kepada negara yang terutang oleh orang pribadi atau badan yang bersifat memaksa berdasarkan undang-undang, dengan tidak mendapatkan imbalan secara langsung dan digunakan untuk keperluan negara bagi sebesar-besarnya kemakmuran rakyat.
multilingual_mpnet_embeddings_raw:
 [-0.00838132  0.12740824 -0.01244919  0.05364892 -0.10382102 -0.06077616
  0.04771158  0.03325832 

In [ ]:
# model_multimpnet_raw.save("/content/drive/MyDrive/Thesis_Finetuned/paraphrase-multilingual-mpnet-base-v2")

# Similarity

## Query & Key Terms

In [ ]:
dfSearch = dfAns[['Bab', 'State', 'Pasal', 'Ayat', 'Angka', 'Huruf', 'Paragraf', 'Isi']].copy()
dfSearch['Isi_lower'] = dfSearch['Isi'].apply(lambda x: x.lower())
dfSearch['Sumber'] = dfSearch.apply(lambda x: set_sumber(x['State'], str(x['Pasal']), str(x['Ayat']), str(x['Angka']), str(x['Huruf']), str(x['Paragraf'])), axis=1)

## SBERT

In [ ]:
def search_query(query, model, embeddings, column_name):
  query_embedding = model.encode(query)
  similarity = util.cos_sim(query_embedding, embeddings)
  index_score = [i.item() for i in similarity[0]]
  dfAns[column_name] = index_score

## BM25

In [ ]:
from rank_bm25 import BM25Okapi

corpus = dfAns['Isi']
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25_isi = BM25Okapi(tokenized_corpus)

corpus = dfAns['Isi_lower']
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

corpus = dfAns['Isi_Tokenized_Joined']
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25_joined = BM25Okapi(tokenized_corpus)

corpus = dfAns['Isi_Stemmed']
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25_stemmed = BM25Okapi(tokenized_corpus)

# Reciprocal Rank Fusion (RRF)

In [ ]:
def reciprocal_rank_fusion(result):
  k = 60
  return sum([1.0 / (k + i) for i in result])

# Evaluation

In [ ]:
def precision_calc(relevant_document, retrieved_document):
  return len(set(relevant_document).intersection(set(retrieved_document))) / len(retrieved_document)

def recall_calc(relevant_document, retrieved_document):
  return len(set(relevant_document).intersection(set(retrieved_document))) / len(relevant_document)

def f_measure_calc(precision, recall):
  precision_recall = precision + recall
  if precision_recall == 0:
    return 0
  else:
    return (2 * precision * recall) / (precision + recall)

def evaluation(df, column_index, key_terms, k):
  match_result = list(df.sort_values(by=column_index)['index_corpus'].head(k))
  precision_score = precision_calc(key_terms, match_result)
  recall_score = recall_calc(key_terms, match_result)
  f_measure_score = f_measure_calc(precision_score, recall_score)
  return precision_score, recall_score, f_measure_score

### End To End Evaluation

In [ ]:
def search_term_list(seris, terms):
  index_term = []
  for term in terms:
    i = 0
    for seri in seris:
      if term in seri:
        index_term.append(i)
      i += 1
  return index_term

dfSearch['Pasal'] = [x.lower() for x in dfSearch['Pasal']]
dfSearch['Ayat'] = [x.lower() for x in dfSearch['Ayat']]
dfSearch['Angka'] = [x.lower() for x in dfSearch['Angka']]
dfSearch['key_answer'] = dfSearch.apply(lambda x: x['Pasal'] + '/' + x['Ayat'] + '/' + x['Angka'], axis=1)

In [ ]:
query = 'Pemberian hukuman untuk pegawai pajak.'
terms = ['pidana']
# key_answer = []
key_answer = ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0']

query_isi = query
query = query.lower()
query_tokenized = text_prepros(query)
query_tokenized = word_tokenize_wrapper(query_tokenized)
query_tokenized_joined = ' '.join(query_tokenized)
query_stemmed = stemmed_wrapper(query_tokenized_joined)

print('query_isi:', query_isi)
print('query:', query)
print('query_tokenized_joined:', query_tokenized_joined)
print('query_stemmed:', query_stemmed)

terms = [term.lower() for term in terms]
print('terms:', terms)

print('##### TERM SEARCH #####')
key_terms = search_term_list(dfSearch['Isi_lower'], terms)
if len(key_answer) > 0:
  key_answer = [answer.lower() for answer in key_answer]
  key_terms = dfSearch[dfSearch['key_answer'].isin(key_answer)].index

query_isi: Pemberian hukuman untuk pegawai pajak.
query: pemberian hukuman untuk pegawai pajak.
query_tokenized_joined: pemberian hukuman untuk pegawai pajak
query_stemmed: beri hukum untuk pegawai pajak
terms: ['pidana']
##### TERM SEARCH #####


In [ ]:
print('##### SBERT SEARCH #####')
dfAns.drop(dfAns.columns[14:], axis=1, inplace=True)

search_query(query = query_isi, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings_isi, column_name = 'multi_mpnet_isi')
search_query(query = query, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings, column_name = 'multi_mpnet')
search_query(query = query_tokenized_joined, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings_joined, column_name = 'mpnet_joined')
search_query(query = query_stemmed, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings_stemmed, column_name = 'mpnet_stemmed')

search_query(query = query_isi, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_isi_raw, column_name = 'mpnet_isi_raw')
search_query(query = query, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_raw, column_name = 'multi_mpnet_raw')
search_query(query = query_tokenized_joined, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_joined_raw, column_name = 'mpnet_joined_raw')
search_query(query = query_stemmed, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_stemmed_raw, column_name = 'mpnet_stemmed_raw')

print('##### BM25 SEARCH #####')
tokenized_query_isi = query_isi.split(" ")
tokenized_query = query.split(" ")
tokenized_query_joined = query_tokenized_joined.split(" ")
tokenized_query_stemmed = query_stemmed.split(" ")

doc_scores = bm25_isi.get_scores(tokenized_query_isi)
dfAns['BM25_isi'] = doc_scores
doc_scores = bm25.get_scores(tokenized_query)
dfAns['BM25'] = doc_scores
doc_scores = bm25_joined.get_scores(tokenized_query_joined)
dfAns['BM25_joined'] = doc_scores
doc_scores = bm25_stemmed.get_scores(tokenized_query_stemmed)
dfAns['BM25_stemmed'] = doc_scores

##### SBERT SEARCH #####
##### BM25 SEARCH #####


In [ ]:
dfAns2 = dfAns.copy()
dfAns2['index_corpus'] = dfAns2.index

dfAns2 = dfAns2.sort_values(by='multi_mpnet_isi', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_mpnet_isi'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='multi_mpnet', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_multi_mpnet'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='mpnet_joined', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_mpnet_joined'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='mpnet_stemmed', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_mpnet_stemmed'] = dfAns2.index + 1

dfAns2 = dfAns2.sort_values(by='mpnet_isi_raw', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_mpnet_isi_raw'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='multi_mpnet_raw', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_multi_mpnet_raw'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='mpnet_joined_raw', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_mpnet_joined_raw'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='mpnet_stemmed_raw', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_mpnet_stemmed_raw'] = dfAns2.index + 1

dfAns2 = dfAns2.sort_values(by='BM25_isi', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_BM25_isi'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='BM25', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_BM25'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='BM25_joined', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_BM25_joined'] = dfAns2.index + 1
dfAns2 = dfAns2.sort_values(by='BM25_stemmed', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_BM25_stemmed'] = dfAns2.index + 1

In [ ]:
print('##### HYBRID SEARCH #####')
dfAns2['RRF_isi'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_isi'], x['index_BM25_isi']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_isi', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_isi'] = dfAns2.index + 1

dfAns2['RRF'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_multi_mpnet'], x['index_BM25']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF'] = dfAns2.index + 1

dfAns2['RRF_joined'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_joined'], x['index_BM25_joined']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_joined', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_joined'] = dfAns2.index + 1

dfAns2['RRF_stemmed'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_stemmed'], x['index_BM25_stemmed']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_stemmed', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_stemmed'] = dfAns2.index + 1

print('##### HYBRID SEARCH ON MPNET RAW #####')
dfAns2['RRF_isi_RAW'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_isi_raw'], x['index_BM25_isi']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_isi_RAW', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_isi_RAW'] = dfAns2.index + 1

dfAns2['RRF_RAW'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_multi_mpnet_raw'], x['index_BM25']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_RAW', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_RAW'] = dfAns2.index + 1

dfAns2['RRF_joined_RAW'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_joined_raw'], x['index_BM25_joined']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_joined_RAW', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_joined_RAW'] = dfAns2.index + 1

dfAns2['RRF_stemmed_RAW'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_stemmed_raw'], x['index_BM25_stemmed']]), axis=1)
dfAns2 = dfAns2.sort_values(by='RRF_stemmed_RAW', ascending=False)
dfAns2.reset_index(drop=True, inplace=True)
dfAns2['index_RRF_stemmed_RAW'] = dfAns2.index + 1

##### HYBRID SEARCH #####
##### HYBRID SEARCH ON MPNET RAW #####


In [ ]:
index_columns = [i for i in dfAns2.columns if 'index' in i ]
index_columns.remove('index_corpus')
print('query:', query)
print('terms:', terms)
# print(index_columns)
display(dfSearch.loc[key_terms, ['State', 'Pasal', 'Ayat', 'Angka', 'Isi_lower']].sort_index())
print('Jumlah Pasal yang ditemukan:', len(key_terms))
k = 10 #len(key_terms)
# k = len(key_terms)

for index_key in index_columns:
  display(dfAns2.sort_values(by=index_key)[['State', 'Pasal', 'Ayat', 'Angka']].head(k))
  print(index_key, ':', evaluation(dfAns2, index_key, key_terms, k))

query: pemberian hukuman untuk pegawai pajak.
terms: ['pidana']


,State,Pasal,Ayat,Angka,Isi_lower
304,Pasal,36a,1,0,pegawai pajak yang karena kelalaiannya atau dengan sengaja menghitung atau menetapkan pajak tidak sesuai dengan ketentuan undang-undang perpajakan dikenai sanksi sesuai dengan ketentuan peraturan perundang-undangan.
305,Pasal,36a,2,0,"pegawai pajak yang dalam melakukan tugasnya dengan sengaja bertindak di luar kewenangannya yang diatur dalam ketentuan peraturan perundang-undangan perpajakan, dapat diadukan ke unit internal departemen keuangan yang berwenang melakukan pemeriksaan dan investigasi dan apabila terbukti melakukannya dikenai sanksi sesuai dengan ketentuan peraturan perundang-undangan."
306,Pasal,36a,3,0,pegawai pajak yang dalam melakukan tugasnya terbukti melakukan pemerasan dan pengancaman kepada wajib pajak untuk menguntungkan diri sendiri secara melawan hukum diancam dengan pidana sebagaimana dimaksud dalam pasal 368 kitab undang-undang hukum pidana.
307,Pasal,36a,4,0,"pegawai pajak yang dengan maksud menguntungkan diri sendiri secara melawan hukum dengan menyalahgunakan kekuasaannya memaksa seseorang untuk memberikan sesuatu, untuk membayar atau menerima pembayaran, atau untuk mengerjakan sesuatu bagi dirinya sendiri, diancam dengan pidana sebagaimana dimaksud dalam pasal 12 undang-undang nomor 31 tahun 1999 tentang pemberantasan tindak pidana korupsi dan perubahannya."
308,Pasal,36a,5,0,"pegawai pajak tidak dapat dituntut, baik secara perdata maupun pidana, apabila dalam melaksanakan tugasnya didasarkan pada iktikad baik dan sesuai dengan ketentuan peraturan perundang-undangan perpajakan."


Jumlah Pasal yang ditemukan: 5


,State,Pasal,Ayat,Angka
0,Pasal,36A,3,0
1,Pasal,36A,4,0
2,Pasal,36A,1,0
300,Pasal,41C,2,0
333,Pasal,41,2,0
207,Pasal,39,3,0
351,Pasal,41C,4,0
321,Pasal,39,2,0
337,Pasal,41,1,0
3,Pasal,36A,5,0


index_mpnet_isi : (0.4, 0.8, 0.5333333333333333)


...

index_RRF_RAW : (0.4, 0.8, 0.5333333333333333)


,State,Pasal,Ayat,Angka
0,Pasal,36A,3,0
2,Pasal,36A,1,0
1,Pasal,36A,4,0
21,Pasal,11,1a,0
3,Pasal,36A,5,0
7,Pasal,36A,2,0
9,Pasal,43,1,0
15,Pasal,36B,2,0
19,Pasal,43A,3,0
17,Pasal,17C,6,0


index_RRF_joined_RAW : (0.5, 1.0, 0.6666666666666666)


,State,Pasal,Ayat,Angka
0,Pasal,36A,3,0
1,Pasal,36A,4,0
2,Pasal,36A,1,0
3,Pasal,36A,5,0
4,Pasal,1,0,32
5,Pasal,25,6,0
6,Pasal,1,0,38
7,Pasal,36A,2,0
8,Pasal,14,2,0
9,Pasal,43,1,0


index_RRF_stemmed_RAW : (0.5, 1.0, 0.6666666666666666)


## ALL Evaluation

In [ ]:
tax_term_dict = {
    'ar': 'account representatives',
    'bpn': 'bukti penerimaan negara',
    'bukper': 'bukti permulaan',
    'but': 'bentuk usaha tetap',
    'dirjen': 'direktur jenderal',
    'ditjen': 'direktorat jenderal',
    'djbc': 'direktorat jenderal bea cukai',
    'djp': 'direktorat jenderal pajak',
    'dpp': 'dasar pengenaan pajak',
    'fpp': 'fungsional pemeriksa pajak',
    'nomor identitas wajib pajak': 'nomor pokok wajib pajak',
    'jo': 'joint operation',
    'kkp': 'kertas kerja pemeriksaan',
    'kmk': 'keputusan menteri keuangan',
    'kpp': 'kantor pelayanan pajak',
    'kppn': 'kantor pelayanan perbendaharaan negara',
    'kso': 'kerja sama operasi',
    'ktp': 'kartu tanda penduduk',
    'kup': 'ketentuan umum dan tatacara perpajakan',
    'lhp': 'laporan hasil pemeriksaan',
    'ms pjk': 'masa pajak',
    'nik': 'nomor induk kependudukan',
    'nomor pajak': 'nomor pokok wajib pajak',
    'nppn': 'norma penghitungan penghasilan neto',
    'npwp': 'nomor pokok wajib pajak',
    'ntpn': 'nomor transaksi penerimaan pajak',
    'op': 'orang pribadi',
    'pbb': 'pajak bumi dan bangunan',
    'phkp': 'penghasilan kena pajak',
    'pjk': 'pajak',
    'pkp': 'pengusaha kena pajak',
    'pmk': 'peraturan menteri keuangan',
    'pp': 'peraturan pemerintah',
    'pph': 'pajak penghasilan',
    'ppn': 'pajak pertambahan nilai',
    'ppnbm': 'pajak penjualan atas barang mewah',
    'ptkp': 'penghasilan tidak kena pajak',
    'sdsn': 'susunan dalam satu naskah',
    'sk': 'surat keputusan',
    'skp': 'surat ketetapan pajak',
    'skpib': 'surat keputusan pemberian imbalan bunga',
    'skpkb': 'surat ketetapan pajak kurang bayar',
    'skpkbt': 'surat ketetapan pajak kurang bayar tambahan',
    'skpkpp': 'surat keputusan penghitungan kelebihan pembayaran pajak',
    'skplb': 'surat ketetapan pajak lebih bayar',
    'skpn': 'surat ketetapan pajak nihil',
    'skppkp': 'surat keputusan pengembalian pendahuluan kelebihan pajak',
    'sp': 'subjek pajak',
    'sp2': 'surat pemberitahuan pemeriksaan',
    'spdn': 'subjek pajak dalam negeri',
    'sphp': 'surat pemberitahuan hasil pemeriksaan',
    'spln': 'subjek pajak luar negeri',
    'spm': 'surat perintah membayar',
    'spmp': 'surat perintah melaksanakan penyitaan',
    'sppt': 'surat pemberitahuan pajak terutang',
    'spt': 'surat pemberitahuan',
    'ssp': 'surat setoran pajak',
    'stp': 'surat tagihan pajak',
    'wbt': 'warisan belum terbagi',
    'wna': 'warga negara asing',
    'wni': 'warga negara indonesia',
    'wp': 'wajib pajak'
}

In [ ]:
queries = [
  ('Pemberian hukuman untuk pegawai pajak', ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0']),
  ('sanksi yang diberikan atas pelanggaran - pelanggaran yang dilakukan pegawai pajak', ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0']),
  ('Jelaskan tentang etika yang mengatur pegawai DJP', ['36b/1/0', '36b/2/0', '36b/3/0']),
  ('kode etik bagi seluruh pegawai DJP', ['36b/1/0', '36b/2/0', '36b/3/0']),
  ('Jelaskan tentang Nomor identitas WP', ['1/0/6', '2/1/0', '2/1a/0', '2/4/0', '2/4a/0', '2/6/0', '2/7/0', '2/10/0', '37a/2/0', '44e/2/0']),
  ('Hak Mendahului Negara di dalam penagihan pajak', ['21/1/0', '21/2/0', '21/3/0', '21/4/0', '21/5/0']),
  ('Negara memiliki Hak Mendahului dalam menagih pajak yang terutang', ['21/1/0', '21/2/0', '21/3/0', '21/4/0', '21/5/0']),
  ('Pemberian Sanksi dalam hal SPT tidak disampaikan', ['3/5a/0', '7/1/0', '13/1/0','38a/0/0','39/1/0']),
  ('Bagaimana melakukan pembetulan SPT tahunan', ['8/1/0', '8/1a/0', '8/2/0', '8/2a/0', '8/6/0']),
  ('kriteria menghapus NPWP', ['2/6/0', '2/7/0', '44e/2/0']),
  ('kriteria WP yang dapat dihapus NPWP nya beserta jangka waktunya', ['2/6/0', '2/7/0', '44e/2/0']),
  ('dalam hal SPT dianggap tidak disampaikan', ['3/7/0', '3/7a/0', '4/4b/0']),
  ('SP dapat mengakui ketidakbenaran dalam pengisian SPT', ['8/3/0', '8/3a/0', '8/4/0', '8/5/0', '17b/5/0', '44a/0/0']),
  ('penggunaan SSP sebagai alat untuk membayar pajak', ['1/0/14','3/5/0','10/1/0','10/1a/0','10/2/0']),
  ('bagaimana jika WP kelebihan membayar pajak', ['1/0/19','11/1/0','11/1a/0','11/2/0','11/3/0','11/4/0','17/3/0','17b/1/0','17b/2/0','17c/1/0','17c/6/0','17d/1/0','17d/2/0','27b/1/0','27b/2/0','27b/3/0']),
  ('SKPN diterbitkan apabila', ['17A/1/0', '17A/2/0', '1/0/18']),
  ('apa yang menjadi dasar penagihan atas pajak yang masih terutang', ['18/1/0','20a/8/0']),
  ('wakil WP dalam memenuhi kewajiban perpajakannya', ['32/1/0', '32/2/0']),
  ('kewajiban bagi pihak lain untuk memberikan data dan informasi yang berkaitan dengan perpajakan', ['35a/1/0', '35a/2/0', '2/10/0', '41c/3/0', '41c/4/0']),
  ('bonus pegawai DJP atas capaian kinerjanya', ['36d/1/0', '36d/2/0', '36d/3/0']),
  ('perbuatan tindak pidana perpajakan', ['1/0/26','1/0/27','1/0/31','1/0/32','13/4/0','17b/1a/0','17b/4/0','17b/5/0','17c/2/0','17c/6/0','22/2/0','34/4/0','34/5/0','35/1/0','35/2/0','36a/3/0','36a/4/0','36a/5/0','38/0/0','39/1/0','39/2/0','39/3/0','39a/0/0','40/0/0','41/1/0','41/2/0','41/3/0','41a/0/0','41b/0/0','41c/1/0','41c/2/0','41c/3/0','41c/4/0','43/1/0','43/2/0','43a/1/0','43a/2/0','43a/3/0','43a/4/0','44/1/0','44/2/0','44/3/0','44a/0/0','44b/1/0','44b/2/0','44b/2a/0','44b/2b/0','44b/2c/0','44c/1/0','44c/2/0','44c/3/0','44d/1/0','44e/2/0']),
  ('peran penyidik pajak', ['1/0/31', '1/0/32', '43a/1a/0', '44/1/0', '44/2/0', '44/3/0', '44/4/0', '44a/0/0']),
  ('pemberlakukan NIK sebagai nomor identitas wajib pajak', ['2/1a/0', '44e/2/0'])
]

In [ ]:
import re

def normalization_query(query):
  for tax_term in tax_term_dict:
    query = re.sub(r"\W"+tax_term+"\W", " "+tax_term_dict[tax_term] + " ", query)
    query = re.sub(r"\A"+tax_term+" ", tax_term_dict[tax_term] + " ", query)
    query = re.sub(r" "+tax_term+"\Z", " " + tax_term_dict[tax_term], query)
    query = re.sub(r"\A"+tax_term+"\Z", tax_term_dict[tax_term], query)
    query = re.sub(r"\W"+tax_term.upper()+"\W", " "+tax_term_dict[tax_term] + " ", query)
    query = re.sub(r"\A"+tax_term.upper()+" ", tax_term_dict[tax_term] + " ", query)
    query = re.sub(r" "+tax_term.upper()+"\Z", " " + tax_term_dict[tax_term], query)
    query = re.sub(r"\A"+tax_term.upper()+"\Z", tax_term_dict[tax_term], query)
  return query

# Deactivate bagian ini untuk pemerosesan tanpa normalisasi
i = 0
while i < len(queries):
  queries[i] = list(queries[i])
  queries[i][0] = normalization_query(queries[i][0])
  queries[i] = tuple(queries[i])
  i += 1

for i in queries:
  print(i)

('Pemberian hukuman untuk pegawai pajak', ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0'])
('sanksi yang diberikan atas pelanggaran - pelanggaran yang dilakukan pegawai pajak', ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0'])
('Jelaskan tentang etika yang mengatur pegawai direktorat jenderal pajak', ['36b/1/0', '36b/2/0', '36b/3/0'])
('kode etik bagi seluruh pegawai direktorat jenderal pajak', ['36b/1/0', '36b/2/0', '36b/3/0'])
('Jelaskan tentang Nomor identitas wajib pajak', ['1/0/6', '2/1/0', '2/1a/0', '2/4/0', '2/4a/0', '2/6/0', '2/7/0', '2/10/0', '37a/2/0', '44e/2/0'])
('Hak Mendahului Negara di dalam penagihan pajak', ['21/1/0', '21/2/0', '21/3/0', '21/4/0', '21/5/0'])
('Negara memiliki Hak Mendahului dalam menagih pajak yang terutang', ['21/1/0', '21/2/0', '21/3/0', '21/4/0', '21/5/0'])
('Pemberian Sanksi dalam hal surat pemberitahuan tidak disampaikan', ['3/5a/0', '7/1/0', '13/1/0', '38a/0/0', '39/1/0'])
('Bagaimana melakukan pembetulan surat pemberitahuan tahuna

In [ ]:
dfSearch['key_answer'] = dfSearch.apply(lambda x: x['Pasal'] + '/' + x['Ayat'] + '/' + x['Angka'], axis=1)

for query in queries:
  query_in = query[1]
  query_in.sort()
  query_out = list(dfSearch.loc[dfSearch['key_answer'].isin(query[1]), 'key_answer'])
  query_out.sort()
  if query_in == query_out:
    print('EQUAL', query)

EQUAL ('Pemberian hukuman untuk pegawai pajak', ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0'])
EQUAL ('sanksi yang diberikan atas pelanggaran - pelanggaran yang dilakukan pegawai pajak', ['36a/1/0', '36a/2/0', '36a/3/0', '36a/4/0', '36a/5/0'])
EQUAL ('Jelaskan tentang etika yang mengatur pegawai direktorat jenderal pajak', ['36b/1/0', '36b/2/0', '36b/3/0'])
EQUAL ('kode etik bagi seluruh pegawai direktorat jenderal pajak', ['36b/1/0', '36b/2/0', '36b/3/0'])
EQUAL ('Jelaskan tentang Nomor identitas wajib pajak', ['1/0/6', '2/1/0', '2/10/0', '2/1a/0', '2/4/0', '2/4a/0', '2/6/0', '2/7/0', '37a/2/0', '44e/2/0'])
EQUAL ('Hak Mendahului Negara di dalam penagihan pajak', ['21/1/0', '21/2/0', '21/3/0', '21/4/0', '21/5/0'])
EQUAL ('Negara memiliki Hak Mendahului dalam menagih pajak yang terutang', ['21/1/0', '21/2/0', '21/3/0', '21/4/0', '21/5/0'])
EQUAL ('Bagaimana melakukan pembetulan surat pemberitahuan tahunan', ['8/1/0', '8/1a/0', '8/2/0', '8/2a/0', '8/6/0'])
EQUAL ('kriteria men

In [ ]:
def search_term_list(seris, terms):
  index_term = []
  for term in terms:
    i = 0
    for seri in seris:
      if term in seri:
        index_term.append(i)
      i += 1
  return index_term

dfSearch['Pasal'] = [x.lower() for x in dfSearch['Pasal']]
dfSearch['Ayat'] = [x.lower() for x in dfSearch['Ayat']]
dfSearch['Angka'] = [x.lower() for x in dfSearch['Angka']]
dfSearch['key_answer'] = dfSearch.apply(lambda x: x['Pasal'] + '/' + x['Ayat'] + '/' + x['Angka'], axis=1)

In [ ]:
def evaluate_queries(queries, k=0):
  evaluation_queries_out = []
  for query_answer in queries:
    query = query_answer[0]
    key_answer = query_answer[1]
    # print(query, '\n', key_answer, '\n', evaluate_query(query, key_answer, k), '\n')
    evaluation_queries_out.append([query, evaluate_query(query, key_answer, k)])
  return evaluation_queries_out

def evaluate_query(query, key_answer, k):
  query_isi = query
  query = query.lower()
  query_tokenized_pre = text_prepros(query)
  query_tokenized = word_tokenize_wrapper(query_tokenized_pre)
  query_tokenized_joined = ' '.join(query_tokenized)
  query_stemmed = stemmed_wrapper(query_tokenized_joined)

  if k == 0:
    print(f'Query:\n query_isi: {query_isi}\n query: {query}\n query_tokenized_pre: {query_tokenized_pre}\n query_tokenized_joined: {query_tokenized_joined}\n query_stemmed: {query_stemmed}\n\n')

  if len(key_answer) > 0:
    key_answer = [answer.lower() for answer in key_answer]
    key_terms = dfSearch[dfSearch['key_answer'].isin(key_answer)].index

  if k <= 0:
    k = len(key_terms)

  dfAns.drop(dfAns.columns[14:], axis=1, inplace=True)

  search_query(query = query_isi, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings_isi, column_name = 'multi_mpnet_isi')
  search_query(query = query, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings, column_name = 'multi_mpnet')
  search_query(query = query_tokenized_joined, model = model_multimpnet, embeddings = multilingual_mpnet_embeddings_joined, column_name = 'mpnet_joined')

  search_query(query = query_isi, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_isi_raw, column_name = 'mpnet_isi_raw')
  search_query(query = query, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_raw, column_name = 'multi_mpnet_raw')
  search_query(query = query_tokenized_joined, model = model_multimpnet_raw, embeddings = multilingual_mpnet_embeddings_joined_raw, column_name = 'mpnet_joined_raw')

  tokenized_query_isi = query_isi.split(" ")
  tokenized_query = query.split(" ")
  tokenized_query_joined = query_tokenized_joined.split(" ")

  doc_scores = bm25_isi.get_scores(tokenized_query_isi)
  dfAns['BM25_isi'] = doc_scores
  doc_scores = bm25.get_scores(tokenized_query)
  dfAns['BM25'] = doc_scores
  doc_scores = bm25_joined.get_scores(tokenized_query_joined)
  dfAns['BM25_joined'] = doc_scores

  dfAns2 = dfAns.copy()
  dfAns2['index_corpus'] = dfAns2.index

  # Original Query
  dfAns2 = dfAns2.sort_values(by='mpnet_isi_raw', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_mpnet_isi_raw'] = dfAns2.index + 1

  dfAns2 = dfAns2.sort_values(by='multi_mpnet_isi', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_mpnet_isi'] = dfAns2.index + 1

  dfAns2 = dfAns2.sort_values(by='BM25_isi', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_BM25_isi'] = dfAns2.index + 1

  dfAns2['RRF_isi_RAW'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_isi_raw'], x['index_BM25_isi']]), axis=1)
  dfAns2 = dfAns2.sort_values(by='RRF_isi_RAW', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_RRF_isi_RAW'] = dfAns2.index + 1

  dfAns2['RRF_isi'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_isi'], x['index_BM25_isi']]), axis=1)
  dfAns2 = dfAns2.sort_values(by='RRF_isi', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_RRF_isi'] = dfAns2.index + 1

  # Lower Preprocessing
  dfAns2 = dfAns2.sort_values(by='multi_mpnet_raw', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_multi_mpnet_raw'] = dfAns2.index + 1

  dfAns2 = dfAns2.sort_values(by='multi_mpnet', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_multi_mpnet'] = dfAns2.index + 1

  dfAns2 = dfAns2.sort_values(by='BM25', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_BM25'] = dfAns2.index + 1

  dfAns2['RRF_raw'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_multi_mpnet_raw'], x['index_BM25']]), axis=1)
  dfAns2 = dfAns2.sort_values(by='RRF_raw', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_RRF_raw'] = dfAns2.index + 1

  dfAns2['RRF'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_multi_mpnet'], x['index_BM25']]), axis=1)
  dfAns2 = dfAns2.sort_values(by='RRF', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_RRF'] = dfAns2.index + 1

  # Lower Preprocessing + Cleaning
  dfAns2 = dfAns2.sort_values(by='mpnet_joined_raw', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_mpnet_joined_raw'] = dfAns2.index + 1

  dfAns2 = dfAns2.sort_values(by='mpnet_joined', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_mpnet_joined'] = dfAns2.index + 1

  dfAns2 = dfAns2.sort_values(by='BM25_joined', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_BM25_joined'] = dfAns2.index + 1

  dfAns2['RRF_joined_raw'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_joined_raw'], x['index_BM25_joined']]), axis=1)
  dfAns2 = dfAns2.sort_values(by='RRF_joined_raw', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_RRF_joined_raw'] = dfAns2.index + 1

  dfAns2['RRF_joined'] = dfAns2.apply(lambda x: reciprocal_rank_fusion([x['index_mpnet_joined'], x['index_BM25_joined']]), axis=1)
  dfAns2 = dfAns2.sort_values(by='RRF_joined', ascending=False)
  dfAns2.reset_index(drop=True, inplace=True)
  dfAns2['index_RRF_joined'] = dfAns2.index + 1

  index_columns = [i for i in dfAns2.columns if 'index' in i ]
  index_columns.remove('index_corpus')
  evaluation_out = []
  for index_key in index_columns:
    evaluation_out.append([index_key, evaluation(dfAns2, index_key, key_terms, k)])
  return evaluation_out

ks = [0, 5, 10]
dfEvaluate = {'k':[], 'Method': [], 'Avg_Precision': [], 'Avg_Recall': [], 'Avg_FMeasure': []}
for k in ks:
  search_evaluations = evaluate_queries(queries, k)

  index_all = {}

  for search_evaluation in search_evaluations:
    term = search_evaluation[0]
    method_evaluations = search_evaluation[1]
    # print(term)
    for method_evaluation in method_evaluations:
      index_evaluation = method_evaluation[0]
      metrix_evaluations = method_evaluation[1]

      if index_evaluation not in index_all:
        index_all[index_evaluation] = [metrix_evaluations]
      else:
        index_all[index_evaluation].append(metrix_evaluations)

  print(f'k yang digunakan: {k} \n')
  for index_method in index_all:
    precision = []
    recall = []
    fmeasure = []

    for method_evaluate in index_all[index_method]:
      precision.append(method_evaluate[0])
      recall.append(method_evaluate[1])
      fmeasure.append(method_evaluate[2])

    dfEvaluate['k'].append(k)
    dfEvaluate['Method'].append(index_method)
    dfEvaluate['Avg_Precision'].append(mean(precision))
    dfEvaluate['Avg_Recall'].append(mean(recall))
    dfEvaluate['Avg_FMeasure'].append(mean(fmeasure))
    # print(f'{index_method}\nAverage Precision: {mean(precision)}, Average Recall: {mean(recall)}, Average F-Measure: {mean(fmeasure)}')
  print('')

dfEvaluate = pd.DataFrame.from_dict(dfEvaluate)
dfEvaluate

Query:
 query_isi: Pemberian hukuman untuk pegawai pajak
 query: pemberian hukuman untuk pegawai pajak
 query_tokenized_pre: pemberian hukuman untuk pegawai pajak
 query_tokenized_joined: pemberian hukuman untuk pegawai pajak
 query_stemmed: beri hukum untuk pegawai pajak


Query:
 query_isi: sanksi yang diberikan atas pelanggaran - pelanggaran yang dilakukan pegawai pajak
 query: sanksi yang diberikan atas pelanggaran - pelanggaran yang dilakukan pegawai pajak
 query_tokenized_pre: sanksi yang diberikan atas pelanggaran pelanggaran yang dilakukan pegawai pajak
 query_tokenized_joined: sanksi yang diberikan atas pelanggaran pelanggaran yang dilakukan pegawai pajak
 query_stemmed: sanksi yang beri atas langgar langgar yang laku pegawai pajak


Query:
 query_isi: Jelaskan tentang etika yang mengatur pegawai direktorat jenderal pajak
 query: jelaskan tentang etika yang mengatur pegawai direktorat jenderal pajak
 query_tokenized_pre: jelaskan tentang etika yang mengatur pegawai direktorat 

,k,Method,Avg_Precision,Avg_Recall,Avg_FMeasure
0,0,index_mpnet_isi_raw,0.471114,0.471114,0.471114
1,0,index_mpnet_isi,0.503179,0.503179,0.503179
2,0,index_BM25_isi,0.297296,0.297296,0.297296
3,0,index_RRF_isi_RAW,0.487021,0.487021,0.487021
4,0,index_RRF_isi,0.543000,0.543000,0.543000
5,0,index_multi_mpnet_raw,0.538026,0.538026,0.538026
6,0,index_multi_mpnet,0.542576,0.542576,0.542576
7,0,index_BM25,0.424022,0.424022,0.424022
8,0,index_RRF_raw,0.596049,0.596049,0.596049
9,0,index_RRF,0.660210,0.660210,0.660210


# =====  END  =====

# Catatan
1. apa sebaiknya hanya pasal saja? atau ditambahkan penjelasannya?
2. Jelaskan lebih rinci mengenai BM25 dan MPNET
3. Apakah perlu menyentuh penjelasan atau hanya pasal saja?

- Tambahkan preprocessing khususnya untuk BM25

https://docs.pinecone.io/guides/data/understanding-hybrid-search

https://learn.microsoft.com/en-us/azure/search/hybrid-search-overview

https://learn.microsoft.com/en-us/azure/search/hybrid-search-ranking

https://learn.microsoft.com/en-us/azure/search/index-similarity-and-scoring

https://www.pinecone.io/learn/hybrid-search-intro/

https://weaviate.io/blog/hybrid-search-explained

https://medium.com/etoai/hybrid-search-combining-bm25-and-semantic-search-for-better-results-with-lan-1358038fe7e6

https://api.python.langchain.com/en/latest/_modules/langchain/retrievers/ensemble.html

https://www.sbert.net/docs/pretrained_models.html

https://www.baeldung.com/cs/sentence-vectors-word2vec